In [0]:
%sql
-- Find the top 10 customers who have generated the highest revenue
select *
from
(
  SELECT 
      customer_skey,
      c.CustomerID,
      concat(c.CustomerFirstName,' ',c.CustomerLastName) as customer_name,
      round(SUM(of.Quantity * of.UnitPrice),2) AS total_revenue,
      RANK() OVER (ORDER BY SUM(of.Quantity * of.UnitPrice) DESC) AS rank_customer
  FROM gold.orders_fact of
  LEFT JOIN gold.customers c
      ON c.CustomerID = of.CustomerID
  GROUP BY 
      customer_skey,
      c.CustomerID,
      concat(c.CustomerFirstName,' ',c.CustomerLastName) 
  ORDER BY total_revenue DESC
)
where 
  rank_customer <= 10

%md
## Part-to-Whole Analysis
### Purpose:
    - To compare performance or metrics across dimensions or time periods.
    - To evaluate differences between categories.
    - Useful for A/B testing or regional comparisons.

In [0]:
%sql
-- Which customer segments contribute the most to overall revenue?
WITH segment_revenue AS (
    SELECT
        customer_segment,
        round(SUM(total_revenue),2) AS total_revenue,
        COUNT(total_products) AS total_products,
        round(SUM(avg_monthly_spend),2) AS total_avg_monthly_spend
    FROM gold.customer_segment
    GROUP BY customer_segment
)
SELECT
    customer_segment,
    total_products,
    total_revenue,
    total_avg_monthly_spend,
    ROUND((CAST(total_revenue AS FLOAT) / SUM(total_revenue) OVER ()) * 100, 2) AS percentage_of_total,
    RANK() OVER (ORDER BY total_revenue DESC) AS rank_customer
FROM segment_revenue;